# Using SQLAlchemy for Python interaction with a database

## Introduction
This notebook introduces using SQLAlchemy to interact with a PostgreSQL database.  It readily applies to MySQL and other relational database products with a simple change to the engine configuration (see db_string).

### Three approaches for using SQLAlchemy
Many developers who are already familiar with Python and SQL will simply install a thin Python wrapper to the database driver and write all the database logic using Python strings to execute raw SQL against the database.  

This notebook outlines the three layers of abstraction at which SQLAlchemy can be used. The first approach presents the simple Python wrapper approach, just mentioned.  The next two approaches present increasing levels of abstraction, with the third approach interacting with our database in a Pythonic way.

### A note about ORM
In the discussion that follows ORM stands for Object-relational mapping, which is a programming technique for converting data between incompatible type systems using object-oriented programming languages. Many popular SQL database products can only store and manipulate scalar values such as integers and strings organized within tables. The programmer must then convert the object values into groups of simpler values for storage in the database (and convert them back upon retrieval).  ORM facilitates this.

### Benfits of SQLAlchemy
SQLAlchemy offers several benefits over the raw SQL approach, including:

* **Cleaner code:** Embedding raw SQL code in Python strings gets messy pretty quickly,
* **More secure code:** Using SQLAlchemy's ORM functionalities can help mitigate against vulnerabilities such as SQL injection,
* **Simpler logic:** SQLAlchemy allows us to abstract all of our database logic into Python objects. Instead of having to think on a table, row, and column level, we can consider everything on a class, instance, and attribute level.

### Caveat
SQLAlchemy interacts with relational databases; it does not support NoSQL databases, such as Cassandra and Mongo.  Two poular software libraries that provide Python support for NoSQL are PyMongo (simple) and MongoEngine (ORM functionality).  See https://realpython.com/introduction-to-mongodb-and-python

#### Prerequisites
1. PostgreSQL database running at localhost port 5432
2. A database named "sqa_test"
3. The password for the database (db_key) is defined in config.py

#### Notes
1. Each snippet for the three approaches is meant to be executed individually (do not run all as one step). 
2. Each snippet creates a table in the database named "films".  Unfortunately, as written, the snippets for Approach # 1 and # 2 will throw an "Duplicate Table" error if that table already exists.  To remedy, use pgAdmin to delete the films table before executing those snippets.  This is not applicalble for Approach # 3.

## Approach # 1: Using Raw SQL to Create, Read, Update, and Delete 
In this section, each of the CRUD operations against PostgreSQL are run through SQLAlchemy, using only raw SQL statements. Using this method, we get none of the advantages mentioned above, and we may as well use only a basic Python database driver (e.g. psycopg, which is the most popular PostgreSQL adapter for the Python programming language and which we need to install along with SQLAlchemy in any case).

In [1]:
# ------------------------------------------------------- #
# Insure that the database does not have a films table.
# This snippet, as written, tries to create a film table,
# and will throw an error if films already exists.
# ------------------------------------------------------- #

from sqlalchemy import create_engine
from config import db_key   # to unlock the database

db_string = "postgres://postgres:" + db_key + "@localhost:5432/sqa_test"

db = create_engine(db_string)

# Create a table and insert one row
db.execute("CREATE TABLE IF NOT EXISTS films (title text, director text, year text)")  
db.execute("INSERT INTO films (title, director, year) VALUES ('1917', 'Sam Mendes', '2019')")

# Read all rows (but there is only one)
result_set = db.execute("SELECT * FROM films")  
for r in result_set:  
    print(f"A row is {r}")

# Update a row
db.execute("UPDATE films SET title='Some2019Film' WHERE year='2019'")

# Delete a row
db.execute("DELETE FROM films WHERE year='2019'") 

A row is ('1917', 'Sam Mendes', '2019')


### Observations of Approach # 1
The above code would become even messier if we were to start using dynamic values, if we needed to run more complicated queries, or if we were dealing with anything more complicated than our single-table toy database.  Even with such a simple case, our database code is already getting messy. The strings are long, and we'd have issues if we needed to insert double or single quotation marks in any of the fields, as we are already using both (double for the Python strings and single for the SQL strings)

## Approach # 2: Using the SQL Expression Language to Create, Read, Update, and Delete
In this section, we'll show how to achieve exactly the same as above, but using the **SQL Expression Language that SQLAlchemy provides,** instead of using raw SQL strings. We'll see that this code is a little bit more verbose, and also fairly complicated and difficult to read, so we don't gain that much over the raw SQL queries above. But we can see that we're already starting to get closer to interacting with our database in a pythonic way.

In [2]:
# ------------------------------------------------------- #
# Insure that the database does not have a films table.
# This snippet, as written, tries to create a film table,
# and will throw an error if films already exists.
# ------------------------------------------------------- #

from sqlalchemy import create_engine  
from sqlalchemy import Table, Column, String, MetaData
from config import db_key   # to unlock the database
db_string = "postgres://postgres:" + db_key + "@localhost:5432/sqa_test"
db = create_engine(db_string)

meta = MetaData(db)  
film_table = Table('films', meta,  
                       Column('title', String),
                       Column('director', String),
                       Column('year', String))

with db.connect() as conn:

    # Create a table and insert one row
    film_table.create()
    insert_statement = film_table.insert().values(title="1917", director="Sam Mendes", year="2019")
    conn.execute(insert_statement)

    # Read all rows (but there is only one)
    select_statement = film_table.select()
    result_set = conn.execute(select_statement)
    for r in result_set:
        print(f"A row is {r}")

    # Update a row
    update_statement = film_table.update().where(film_table.c.year=="2019").values(title = "Some2019Film")
    conn.execute(update_statement)

    # Delete a row
    delete_statement = film_table.delete().where(film_table.c.year == "2019")
    conn.execute(delete_statement)
    

A row is ('1917', 'Sam Mendes', '2019')


### Observations of Approach # 2
Note a few more imports were added at the top of this snippet to allow us to talk about the same concepts as before (Table, Column and String) using Python instead of SQL. The biggest change is creating a Table as a Python class, which takes a variable number of Column objects as arguments, and then calling .create() on this to actually create the table in Postgres.

Our four CRUD operations are a direct parallel of what we did before, but instead of writing the statement as a long string, we can chain together various functions such as update(), where(), and values() that are provided by SQLAlchemy's SQL Expression Language.

## Approach # 3: Using the SQL ORB to Create, Read, Update, and Delete
In this last section, we'll see how to do the same thing as in the previous examples using the full ORM. Our code is now longer again in terms of lines, but it feels more concise in terms of line length, and it is more readable (once you're used to the concepts behind the ORM). At this point, we can largely ignore database concepts such as tables, and think only in terms of Python objects.

In [3]:
# ------------------------------------------------------------ #
# You don't need to remember to manually drop the films table.
# ------------------------------------------------------------ #

from sqlalchemy import create_engine  
from sqlalchemy import Column, String  
from sqlalchemy.ext.declarative import declarative_base  
from sqlalchemy.orm import sessionmaker
from config import db_key   # to unlock the database

db_string = "postgres://postgres:" + db_key + "@localhost:5432/sqa_test"

db = create_engine(db_string)  
base = declarative_base()

class Film(base):  
    __tablename__ = 'films'

    title = Column(String, primary_key=True)
    director = Column(String)
    year = Column(String)

# Perform the hidden SQLAlchemy magic
Session = sessionmaker(db)  
session = Session()

# Send a CREATE TABLE command to the database, using the schema that was 
# implicitly created by declaring the Film class
base.metadata.create_all(db)

# Create one row
doctor_strange = Film(title="1917", director="Sam Mendes", year="2019")  
session.add(doctor_strange)  
session.commit()

# Read all rows (but there is only one)
films = session.query(Film)  
for film in films:  
    print(f'The title of a film is "{film.title}"')

# Update a row
doctor_strange.title = "Some2019Film"  
session.commit()

# Delete a row
session.delete(doctor_strange)  
session.commit()  

The title of a film is "1917"


### Observations of Approach # 3
There are many things happening under the hood in this example, but here are some highlights:
* Unlike was the case for the first two approaches, we do not need to worry about whether a table already exists int the database.  SQLAlchemy watches for this.
* The Film class is a subclass of the base class, which SQLAlchemy provides and in turn provides many database shortcuts.
* You can think of a session as an intelligent connection to our database that will watch what we're doing in our Python code and modify the database as necessary.
* We can insert data into our database by instantiating Python objects and passing them to session.add(). In this example, doctor_strange is an object of the Film class.
* Perhaps the starkest difference is in our update example. Here we don't pass anything to the session at all — we simply modify our Python object, and then call session.commit(), which notices the modification and makes the update call to the database.

## Conclusion
So what is the catch in using a SQLAlchemy abstraction layer to implement object-oriented progamming techniques for interacting with a database?  Well, you might assume that there can be a hit to performance, which would be noticeable for very, very large databases.  

However, databases seldom exist in a vacuum -- outboard programs must be used to do useful things with those databases (web pages, visualization, analysis).  Earlier techniques sent raw SQL commands to the database using a Python wrapper, which may be appropriate for throw-away code (non-maintained) or when existing SQL code must be used.  This approach "may" also affect performance less than when using an ORM.

The ORM is useful for keeping your code Pythonic and to abstract away from the database completely. It allows one to deal directly with Python classes instead of Tables, with instantiations of objects instead of rows, and with object attributes instead of columns. It also abstracts away from the specific database flavor of the day that you're using, as SQLAlchemy has drivers for popular database products.